In [2]:
import numpy as np

In [34]:
a = np.ones((2,))
a[1]  =3
a

array([1., 3.])

In [35]:
1 / (1.0 + np.exp(-a))

array([0.73105858, 0.95257413])

In [36]:
b = a * (1 - a)
b

array([ 0., -6.])

In [37]:
b[1]

-6.0

In [39]:
h = np.ones((2,))
h[1]  = 7
h

array([1., 7.])

In [40]:
b * h 

array([  0., -42.])

In [41]:
b[0] * h 

array([0., 0.])

In [42]:
b[1] * h 

array([ -6., -42.])

In [43]:
grad = np.ones((2,2))
grad

array([[1., 1.],
       [1., 1.]])

In [44]:
grad[:, 1]

array([1., 1.])

In [56]:
grad[:, 0] = b[0] * h 
grad[:, 1] = b[1] * h 
grad = np.ones((2,2))

grad[0, :] =  h 
grad[1, :] =  h 

In [57]:
grad

array([[1., 7.],
       [1., 7.]])

In [54]:
grad = np.ones((2,2))

grad[:, 0] =  h 
grad[:, 1] =  h 

In [55]:
grad # 코드 

array([[1., 1.],
       [7., 7.]])

In [59]:
grad[0] * h

array([ 1., 49.])

In [30]:
b[0] * h *np.ones((2,1))

array([[0.],
       [0.]])

In [ ]:
a = np.ones((2, 3))
a

In [ ]:
a[:, 1] = 2

In [ ]:
a

array([[1., 2., 1., 1.],
       [1., 2., 1., 1.],
       [1., 2., 1., 1.]])

In [ ]:
a[:, 3] = 2
a

array([[1., 2., 1., 2.],
       [1., 2., 1., 2.],
       [1., 2., 1., 2.]])

In [ ]:
a[0, :] = ([9,9,9])

In [ ]:
a

array([[9., 9., 9.],
       [1., 1., 1.]])

In [ ]:
a.shape

(2, 3)

## 역전파 학습법을 이용한 심층 신경망 학습

In [ ]:
import time
import numpy as np

## 유틸리티 함수

In [ ]:
def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

## Sigmoid 구현

In [ ]:
class Sigmoid:
    def __init__(self):
        self.last_o = 1

    def __call__(self, x):  # 1 / 1 + np.exp(-x)
        self.last_o = 1 / (1.0 + np.exp(-x))
        return self.last_o

    def grad(self): # sigmoid(x) * ( 1 - sigmoid(x)), last_o = sigmoid(x)
        return self.last_o * (1 - self.last_o)

## Mean Squared Error 구현

In [ ]:
class MeanSquaredError:
    def __init__(self):
        self.dh = 1 #???gradient L(self).dh = L.grad() #dL/dh
        self.diff = 1

    def __call__(self, h, y): # 1/2 * np.mean(np.sqrt(np.abs(h, y)))   1/2*m(h-y)^2
        self.diff = h - y
        return 1/2 * np.mean(np.square(h - y))

    def grad(self): # h - y
        return self.diff

## 뉴런 구현

In [ ]:
class Neuron:
    def __init__(self, W, b, a_obj):
        self.W = W
        self.b = b
        self.a = a_obj()

        #gradient
        self.dh = np.zeros_like(_t(self.W)) #chain rule
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)

        self.last_x = np.zeros((self.W.shape[0])) #(5,) 
        self.last_h = np.zeros((self.W.shape[1])) #(10,)

    def __call__(self, x):
        self.last_x = x
        self.last_h = _m(_t(self.W), x) + self.b
        return self.a(self.last_h)

    def grad(self): #chain rule df(hn-1) /dhn-1 =W 
        
        return self.W * self.a.grad()

    def grad_W(self, dh): #h(n_1) x -> last_x , dh 
        grad = np.ones_like(self.W) # j 반복 W[:, j] = h_n 
        grad_a = self.a.grad()
        for j in range(grad.shape[1]):
          grad[:, j] = dh[j] * grad_a[j] * self.last_x
        return grad 

    def grad_b(self, dh): # 1
        return dh * self.a.grad()

## 심층신경망 구현

In [ ]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, input, output, activation=Sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))

        # Hidden Layers
        for index in range(hidden_depth):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))

        # Output Layer
        W, b = init_var(num_neuron, output)
        self.sequence.append(Neuron(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, loss_obj):
        loss_obj.dh = loss_obj.grad()
        self.sequence.append(loss_obj) #Error Backpropagation Algorithm의 sequence L0 dL/dh(last_h)
        for i in range(len(self.sequence)-1, 0, -1): #거꾸로 back propagation
          #i layer [5,4,3,2,1]
          l1 = self.sequence[i] #l1.dh <- loss_obj #이전 layer
          l0 = self.sequence[i - 1] #현 layer

          l0.dh = _m(l0.grad(), l1.dh)
          l0.dw = l0.grad_W(l1.dh)
          l0.db = l0.grad_b(l1.dh)
          
          #loss의  gradient
          # 미리 call 되어, y_hat(network(X)) y - loss_obj(y_hat, y) .grad()  #mean squared error (h, y) -> grad() h - y

          #chain_rule layer_dh = w <- Fn(hn-1) <- hn-1로 미분
          #저장 되어야 - dh 
        
          #fn(hn-1) <- Wn으로 미분 , hn-1 =x
          #layer.dW = layer.grad_W(dh)
          #fn(hn-1) <- bn으로 미분 , 1
          #layer.db = layer.grad_b()

        self.sequence.remove(loss_obj) # NN의 layer로 


## 경사하강 학습법

In [ ]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = loss_obj(network(x), y)  # Forward inference
    network.calc_gradient(loss_obj)  # Back-propagation
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [ ]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth=5, num_neuron=32, input=10, output=2, activation=Sigmoid)
loss_obj = MeanSquaredError()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, loss_obj, alpha=0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.24196689924520007
Epoch 1: Test loss 0.24166450688344335
Epoch 2: Test loss 0.24136248592614082
Epoch 3: Test loss 0.24106083773618103
Epoch 4: Test loss 0.24075956366470774
Epoch 5: Test loss 0.24045866505111166
Epoch 6: Test loss 0.24015814322302417
Epoch 7: Test loss 0.23985799949631043
Epoch 8: Test loss 0.23955823517506455
Epoch 9: Test loss 0.23925885155160517
Epoch 10: Test loss 0.23895984990647107
Epoch 11: Test loss 0.23866123150841898
Epoch 12: Test loss 0.2383629976144208
Epoch 13: Test loss 0.23806514946966245
Epoch 14: Test loss 0.2377676883075431
Epoch 15: Test loss 0.2374706153496753
Epoch 16: Test loss 0.23717393180588564
Epoch 17: Test loss 0.23687763887421684
Epoch 18: Test loss 0.23658173774092958
Epoch 19: Test loss 0.23628622958050563
Epoch 20: Test loss 0.23599111555565214
Epoch 21: Test loss 0.23569639681730573
Epoch 22: Test loss 0.23540207450463801
Epoch 23: Test loss 0.23510814974506136
Epoch 24: Test loss 0.23481462365423592
Epoch 25: Tes